In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

data = [
    {"customer_id":1001, "name":"Asha Mehta",  "phone":"+91-9876543210", "address":"23 Lodi Rd, Delhi",   "plan_type":"Prepaid",  "arpu_inr":180},
    {"customer_id":1002, "name":"Ravi Kumar",  "phone":"+91-9988776655", "address":"12 Marine Dr, Mumbai","plan_type":"Postpaid", "arpu_inr":280},
    {"customer_id":1003, "name":"Sneha Rao",   "phone":"+91-9090909090", "address":"7 Anna Salai, Chennai","plan_type":"Prepaid", "arpu_inr":210},
    {"customer_id":1004, "name":"Manoj Singh", "phone":"+91-9123456789", "address":"44 CR Park, Delhi",   "plan_type":"Postpaid", "arpu_inr":320},
    {"customer_id":1005, "name":"Divya Jain",  "phone":"+91-9000011111", "address":"18 Park St, Kolkata", "plan_type":"Prepaid",  "arpu_inr":120}
]
pd.DataFrame(data).to_csv("customer_data.csv", index=False)
print("✅ customer_data.csv created")



✅ customer_data.csv created


In [5]:

import pandas as pd
from datetime import datetime
import os

AUDIT_LOG = "access_audit.log"

def log_access(role: str, fields_returned: list):
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(AUDIT_LOG, "a", encoding="utf-8") as f:
        f.write(f"[{ts}] role={role} fields={fields_returned}\n")

# ---- Role policy (easy to read/modify) ----
#   - "mask" => partial masking
#   - "hide" => fully redacted
#   - "show" => no change
POLICY = {
    "junior_analyst": {
        "name":   "hide",
        "phone":  "hide",
        "address":"hide",
        "arpu_inr":"show",
        "plan_type":"show"
    },
    "senior_analyst": {
        "name":   "hide",
        "phone":  "mask",     # keep only first 3 digits, mask rest
        "address":"hide",
        "arpu_inr":"show",
        "plan_type":"show"
    },
    "manager": {
        "name":   "show",
        "phone":  "show",
        "address":"show",
        "arpu_inr":"show",
        "plan_type":"show"
    }
}

def apply_masking(df: pd.DataFrame, role: str) -> pd.DataFrame:
    if role not in POLICY:
        raise ValueError(f"Unknown role: {role}")

    pol = POLICY[role]
    out = df.copy()

    # NAME
    if pol.get("name") == "hide" and "name" in out:
        out["name"] = "ANONYMIZED"

    # PHONE
    if "phone" in out:
        if pol.get("phone") == "hide":
            out["phone"] = "XXXXX"
        elif pol.get("phone") == "mask":
            # keep first 3 digits/characters; mask the rest uniformly
            out["phone"] = out["phone"].astype(str).str[:3] + "-XXX-XXXX"

    # ADDRESS
    if pol.get("address") == "hide" and "address" in out:
        out["address"] = "HIDDEN"

    # Everything else is passed through (“show”)
    return out

def get_customer_data(role: str, fields: list | None = None) -> pd.DataFrame:
    """Return a role-appropriate view. Optional `fields` lets you choose columns."""
    df = pd.read_csv("customer_data.csv")

    # Always keep stable technical columns (id)
    mandatory = ["customer_id"]
    if fields is None:
        fields = list(df.columns)  # start with all
    fields = list(dict.fromkeys(mandatory + fields))  # ensure unique, keep order
    df = df[fields]

    masked = apply_masking(df, role)
    log_access(role, fields)
    return masked


In [6]:
cols = ["customer_id", "name", "phone", "address", "plan_type", "arpu_inr"]

print(" Junior Analyst view:")
jun = get_customer_data("junior_analyst", cols)
display(jun.head())
jun.to_csv("view_junior.csv", index=False)

print("\n Senior Analyst view:")
sen = get_customer_data("senior_analyst", cols)
display(sen.head())
sen.to_csv("view_senior.csv", index=False)

print("\n Manager view:")
mgr = get_customer_data("manager", cols)
display(mgr.head())
mgr.to_csv("view_manager.csv", index=False)

print("\n Audit Log preview:")
print(open(AUDIT_LOG, "r", encoding="utf-8").read().splitlines()[-3:])



 Junior Analyst view:


,customer_id,name,phone,address,plan_type,arpu_inr
0,1001,ANONYMIZED,XXXXX,HIDDEN,Prepaid,180
1,1002,ANONYMIZED,XXXXX,HIDDEN,Postpaid,280
2,1003,ANONYMIZED,XXXXX,HIDDEN,Prepaid,210
3,1004,ANONYMIZED,XXXXX,HIDDEN,Postpaid,320
4,1005,ANONYMIZED,XXXXX,HIDDEN,Prepaid,120



 Senior Analyst view:


,customer_id,name,phone,address,plan_type,arpu_inr
0,1001,ANONYMIZED,+91-XXX-XXXX,HIDDEN,Prepaid,180
1,1002,ANONYMIZED,+91-XXX-XXXX,HIDDEN,Postpaid,280
2,1003,ANONYMIZED,+91-XXX-XXXX,HIDDEN,Prepaid,210
3,1004,ANONYMIZED,+91-XXX-XXXX,HIDDEN,Postpaid,320
4,1005,ANONYMIZED,+91-XXX-XXXX,HIDDEN,Prepaid,120



 Manager view:


,customer_id,name,phone,address,plan_type,arpu_inr
0,1001,Asha Mehta,+91-9876543210,"23 Lodi Rd, Delhi",Prepaid,180
1,1002,Ravi Kumar,+91-9988776655,"12 Marine Dr, Mumbai",Postpaid,280
2,1003,Sneha Rao,+91-9090909090,"7 Anna Salai, Chennai",Prepaid,210
3,1004,Manoj Singh,+91-9123456789,"44 CR Park, Delhi",Postpaid,320
4,1005,Divya Jain,+91-9000011111,"18 Park St, Kolkata",Prepaid,120



 Audit Log preview:
["[2025-10-29 20:27:36] role=junior_analyst fields=['customer_id', 'name', 'phone', 'address', 'plan_type', 'arpu_inr']", "[2025-10-29 20:27:36] role=senior_analyst fields=['customer_id', 'name', 'phone', 'address', 'plan_type', 'arpu_inr']", "[2025-10-29 20:27:36] role=manager fields=['customer_id', 'name', 'phone', 'address', 'plan_type', 'arpu_inr']"]
